In [2]:
import numpy as np
from scipy import stats
import seaborn as sns
import pandas as pd

In [1]:
def get_pearson_matrix(raw, channels): 
    ch = channels
    pearson = {} 

    for c in ch: 
        pearson[c] = {} 
        pearson[c][c] = 1
        
    _ch = ch.copy()
    while(len(_ch) > 0):
        chl1 = _ch.pop(0) 
        for chl2 in _ch:
            raw1, _ = raw[chl1]
            raw2, _ = raw[chl2]

            pearson_c = stats.pearsonr(raw1.flatten(), raw2.flatten())

            pearson[chl1][chl2] = pearson_c[0]
            pearson[chl2][chl1] = pearson_c[0]
    return pearson 


In [8]:
df = pd.read_excel("Demographic_List.xlsx")

df.head(10)

,sub,Age,Gender,Study,count,session,Condition,task,Headset
0,9003,41,F,INSPECDS,286,900301000,HC,RSEC,X24
1,9003,41,F,INSPECDS,296,900301000,HC,RSEO,X24
2,9004,66,F,INSPECDS,266,900401000,HC,RSEC,X24
3,9004,66,F,INSPECDS,285,900401000,HC,RSEO,X24
4,9006,52,M,INSPECDS,288,900601000,HC,RSEC,X24
5,9006,52,M,INSPECDS,291,900601000,HC,RSEO,X24
6,9008,53,F,INSPECDS,291,900801000,HC,RSEC,X24
7,9008,53,F,INSPECDS,290,900801000,HC,RSEO,X24
8,9009,48,F,INSPECDS,292,900901000,HC,RSEC,X24
9,9009,48,F,INSPECDS,279,900901000,HC,RSEO,X24


In [12]:
conditions = set(df["Condition"])
conditions

{'AD', 'HC', 'MCI'}

In [14]:
patients_id = set(df["sub"])